# 4. Model Deployment (PKL to ONNX) - Plank

**Objective:**
1. Load a specific trained pipeline (e.g., RandomForest) from `.pkl`.
2. Convert the entire `scikit-learn` pipeline to ONNX format using `ModelDeployConverterUtils`.
3. Save the ONNX model to the `models/onnx/` directory.

**Note:** We are converting the *entire pipeline* (imputer + scaler + model). Therefore, we do **not** need to convert the scaler to a separate JSON file. The ONNX model will handle all preprocessing steps.

In [5]:
import os
import sys
import logging
import onnxruntime as ort

module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.model_deploy_convertor_utils import ModelDeployConverterUtils

# Configure logger for the utility to see detailed output
logger = logging.getLogger('utils.model_deploy_convertor_utils')
logger.setLevel(logging.INFO)

## 4.1 Define Paths and Parameters

In [6]:
MODEL_PKL_DIR = "../models/pkl/"
MODEL_ONNX_DIR = "../models/onnx/"
os.makedirs(MODEL_ONNX_DIR, exist_ok=True)

# >>>>> YOU CAN CHANGE THIS VALUE TO 'LR', 'KNN', 'DT', 'RF', or 'XGB' if you saved it earlier <<<<<
MODEL_KEY_TO_CONVERT = 'RF'

# The name of the PKL file without the extension
PKL_MODEL_NAME = f"{MODEL_KEY_TO_CONVERT}_model"

TARGET_OPSET = 12

print(f"Will convert '{PKL_MODEL_NAME}.pkl' to '{PKL_MODEL_NAME}.onnx'")

Will convert 'RF_model.pkl' to 'RF_model.onnx'


## 4.2 Perform Conversion

In [7]:
print(f"--- Starting ONNX Conversion for {PKL_MODEL_NAME} ---")

onnx_model_path = ModelDeployConverterUtils.convert_model_pickle_to_onnx(
    pickle_model_dir=MODEL_PKL_DIR,
    onnx_model_dir=MODEL_ONNX_DIR,
    name=PKL_MODEL_NAME, # The name of the .pkl file (without extension)
    target_opset=TARGET_OPSET
)

if onnx_model_path and os.path.exists(onnx_model_path):
    print(f"\nSUCCESS: Model converted and saved to: {onnx_model_path}")
else:
    print(f"\nFAILURE: Model conversion failed. Check logs above for errors.")

2025-06-08 14:30:47,440 - utils.model_deploy_convertor_utils - INFO - Starting conversion for model 'RF_model': ../models/pkl/RF_model.pkl -> ../models/onnx/RF_model.onnx
2025-06-08 14:30:47,445 - utils.model_deploy_convertor_utils - INFO - Successfully loaded pickle model from ../models/pkl/RF_model.pkl
2025-06-08 14:30:47,447 - utils.model_deploy_convertor_utils - INFO - Model for 'RF_model' expects 20 input features.
2025-06-08 14:30:47,448 - utils.model_deploy_convertor_utils - INFO - Attempting to convert model 'RF_model' to ONNX with target_opset=12.


--- Starting ONNX Conversion for RF_model ---


2025-06-08 14:30:47,723 - utils.model_deploy_convertor_utils - INFO - Successfully converted model 'RF_model' to ONNX format.
2025-06-08 14:30:47,731 - utils.model_deploy_convertor_utils - INFO - Successfully saved ONNX model to ../models/onnx/RF_model.onnx



SUCCESS: Model converted and saved to: ../models/onnx/RF_model.onnx


## 4.3 Verify ONNX Model (Optional)

In [8]:
if onnx_model_path and os.path.exists(onnx_model_path):
    try:
        ort_session = ort.InferenceSession(onnx_model_path)
        input_info = ort_session.get_inputs()[0]
        output_info = ort_session.get_outputs()
        
        print("\n--- ONNX Model Verification ---")
        print(f"Model loaded successfully from {onnx_model_path}")
        print(f"Input Name: {input_info.name}")
        print(f"Input Shape: {input_info.shape}")
        print(f"Input Type: {input_info.type}")
        print(f"Number of Outputs: {len(output_info)}")
        print(f"Output 0 (Labels) Name: {output_info[0].name}, Shape: {output_info[0].shape}")
        print(f"Output 1 (Probabilities) Name: {output_info[1].name}, Shape: {output_info[1].shape}")
    except Exception as e:
        print(f"\nError during ONNX model verification: {e}")
else:
    print("\nSkipping ONNX verification because the model file was not created.")


--- ONNX Model Verification ---
Model loaded successfully from ../models/onnx/RF_model.onnx
Input Name: float_input
Input Shape: [None, 20]
Input Type: tensor(float)
Number of Outputs: 2
Output 0 (Labels) Name: output_label, Shape: [None]
Output 1 (Probabilities) Name: output_probability, Shape: []
